# TNM stage lung cancer

In [ ]:
# %pip install -U graphviz
# %pip install -U langgraph langchain langchain_openai langchain_experimental langchain-community

In [ ]:
from graphviz import Digraph

def create_lung_cancer_t_staging_diagram():
    dot = Digraph(comment='Lung Cancer T Staging Process', format='png')
    dot.attr(rankdir='TB', size='12,16', dpi='300', bgcolor='white')
    
    # 전체 그래프 스타일
    dot.attr('graph', fontname='Arial', fontsize='18', pad='0.5')
    
    # 노드 스타일
    dot.attr('node', shape='rectangle', style='filled,rounded', color='#555555', 
             fontname='Arial', fontsize='14', margin='0.3,0.2', height='0.6')
    
    # 엣지 스타일
    dot.attr('edge', color='#888888', fontname='Arial', fontsize='12', 
             penwidth='1.5', arrowsize='0.9', arrowhead='vee')

    # 서브그래프 및 노드 생성
    with dot.subgraph(name='cluster_0') as c:
        c.attr(label='Initial Analysis', style='filled', fillcolor='#F0F0F0:#E0E0E0', gradientangle='270', color='#D0D0D0')
        c.node('CT_Analyzer', 'CT Analyzer', fillcolor='#FFC3A0:#FF9A8C', gradientangle='90')
        c.node('PET_Analyzer', 'PET Analyzer', fillcolor='#A0FFC3:#8CFF9A', gradientangle='90')
        c.node('Size_Classifier', 'Size-Based Classifier', fillcolor='#A0C3FF:#8C9AFF', gradientangle='90')

    with dot.subgraph(name='cluster_1') as c:
        c.attr(label='Detailed Assessment', style='filled', fillcolor='#F0F0F0:#E0E0E0', gradientangle='270', color='#D0D0D0')
        c.node('Sync_Multi_Checker', 'Synchronous/\nMulticentric Checker', fillcolor='#FFA0FF:#FF8CFF', gradientangle='90')
        c.node('Main_Bronchus_Checker', 'Main Bronchus Checker', fillcolor='#FFD700:#FFA500', gradientangle='90')
        c.node('Invasion_Checker', 'Invasion Checker', fillcolor='#00CED1:#1E90FF', gradientangle='90')

    with dot.subgraph(name='cluster_2') as c:
        c.attr(label='Stage Classification', style='filled', fillcolor='#F0F0F0:#E0E0E0', gradientangle='270', color='#D0D0D0')
        c.node('T_Stage_Classifier', 'T Stage Classifier', fillcolor='#98FB98:#32CD32', gradientangle='90')
        c.node('Year_Checker', 'Year Checker', fillcolor='#DDA0DD:#9370DB', gradientangle='90')
        c.node('Final_T_Stage', 'Final T Stage Determiner', fillcolor='#F08080:#CD5C5C', gradientangle='90')

    dot.node('Result_Validator', 'Result Validator', fillcolor='#20B2AA:#008B8B', gradientangle='90')

    # 엣지 생성
    dot.edge('CT_Analyzer', 'PET_Analyzer')
    dot.edge('PET_Analyzer', 'Size_Classifier')
    dot.edge('Size_Classifier', 'Sync_Multi_Checker')
    dot.edge('Sync_Multi_Checker', 'Main_Bronchus_Checker')
    dot.edge('Main_Bronchus_Checker', 'Invasion_Checker')
    dot.edge('Invasion_Checker', 'T_Stage_Classifier')
    dot.edge('T_Stage_Classifier', 'Year_Checker')
    dot.edge('Year_Checker', 'Final_T_Stage')
    dot.edge('Final_T_Stage', 'Result_Validator')

    # 수퍼바이저 노드 및 엣지
    dot.node('Supervisor', 'Supervisor', shape='diamond', fillcolor='#FFD700:#DAA520', gradientangle='90')
    for node in ['CT_Analyzer', 'PET_Analyzer', 'Size_Classifier', 'Sync_Multi_Checker', 
                 'Main_Bronchus_Checker', 'Invasion_Checker', 'T_Stage_Classifier', 
                 'Year_Checker', 'Final_T_Stage', 'Result_Validator']:
        dot.edge('Supervisor', node, style='dashed', constraint='false', dir='both')

    return dot

# 다이어그램 생성 및 저장
diagram = create_lung_cancer_t_staging_diagram()
diagram.render('lung_cancer_t_staging_process', view=True, cleanup=True)
print("Lung Cancer T Staging Process diagram has been saved and opened.")

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
import operator
from typing import Annotated, TypedDict, Sequence
import functools

class LungCancerTStageState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    ct_result: dict
    pet_result: dict
    tumor_size: float
    tumor_location: str
    main_bronchus_involvement: bool
    synchronous_multicentric: bool
    invasion_details: dict
    initial_t_stage: str
    t_stage: str
    year: int
    final_t_stage: str
    next: str
    explanation: str
    supervisor_approval: bool
    current_node: str

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def supervisor_node(state: LungCancerTStageState, agent):
    """
    수퍼바이저: 에이전트의 판단과 설명을 검토하고 승인 여부를 결정합니다.
    """
    result = agent.invoke({
        "explanation": state['explanation'],
        "current_state": state
    })
    state['supervisor_approval'] = result['output']['approval']
    if not state['supervisor_approval']:
        state['next'] = state['current_node']  # 승인되지 않으면 현재 노드를 재실행
    return state

def wrap_node_with_supervisor(node_func, node_name):
    def wrapped_node(state: LungCancerTStageState, agent, supervisor_agent):
        state['current_node'] = node_name
        result = node_func(state, agent)
        state['explanation'] = result['explanation']
        return supervisor_node(state, supervisor_agent)
    return wrapped_node

def ct_analyzer_node(state: LungCancerTStageState, agent):
    """
    CT 분석기: 종양의 크기와 위치를 정확히 측정합니다.
    """
    result = agent.invoke(state)
    state['ct_result'] = result['output']
    state['tumor_size'] = result['output'].get('tumor_size')
    state['tumor_location'] = result['output'].get('tumor_location')
    if result['output'].get('t_stage_visible') == False:
        state['t_stage'] = 'Tx'
    state['next'] = 'PET_Analyzer'
    state['explanation'] = result['explanation']
    return state

def pet_analyzer_node(state: LungCancerTStageState, agent):
    """
    PET 분석기: 종양의 대사 활성도, 크기, 잠재적 전이를 확인합니다.
    """
    result = agent.invoke(state)
    state['pet_result'] = result['output']
    pet_tumor_size = result['output'].get('tumor_size')
    state['tumor_size'] = max(state['tumor_size'], pet_tumor_size)
    state['next'] = 'Size_Based_Classifier'
    state['explanation'] = result['explanation']
    return state

def size_based_classifier_node(state: LungCancerTStageState, agent):
    """
    크기 기반 분류기: 종양 크기를 기준으로 초기 T 병기를 분류합니다.
    """
    result = agent.invoke(state)
    state['initial_t_stage'] = result['output']
    state['next'] = 'Synchronous_Multicentric_Checker'
    state['explanation'] = result['explanation']
    return state

def synchronous_multicentric_checker_node(state: LungCancerTStageState, agent):
    """
    동시성/다중심성 확인기: 동시에 여러 부위에 암이 있는지 확인합니다.
    """
    result = agent.invoke(state)
    state['synchronous_multicentric'] = result['output']
    state['next'] = 'Main_Bronchus_Checker'
    state['explanation'] = result['explanation']
    return state

def main_bronchus_checker_node(state: LungCancerTStageState, agent):
    """
    주기관지 침범 확인기: 종양이 주기관지를 침범했는지 확인합니다.
    """
    result = agent.invoke(state)
    state['main_bronchus_involvement'] = result['output']
    state['next'] = 'Invasion_Checker'
    state['explanation'] = result['explanation']
    return state

def invasion_checker_node(state: LungCancerTStageState, agent):
    """
    침범 확인기: 종양이 주변 구조물을 침범했는지 상세히 확인합니다.
    """
    result = agent.invoke(state)
    state['invasion_details'] = result['output']
    state['next'] = 'T_Stage_Classifier'
    state['explanation'] = result['explanation']
    return state

def t_stage_classifier_node(state: LungCancerTStageState, agent):
    """
    T 병기 분류기: 수집된 모든 정보를 바탕으로 잠정적인 T 병기를 결정합니다.
    """
    result = agent.invoke(state)
    state['t_stage'] = result['output']
    state['next'] = 'Year_Checker'
    state['explanation'] = result['explanation']
    return state

def year_checker_node(state: LungCancerTStageState, agent):
    """
    연도 확인기: 2018년 전후의 기준을 적용할지 결정합니다.
    """
    result = agent.invoke(state)
    state['year'] = result['output']
    state['next'] = 'Final_T_Stage_Determiner'
    state['explanation'] = result['explanation']
    return state

def final_t_stage_determiner_node(state: LungCancerTStageState, agent):
    """
    최종 T 병기 결정기: 모든 정보와 적용 연도를 고려하여 최종 T 병기를 결정합니다.
    """
    result = agent.invoke(state)
    state['final_t_stage'] = result['output']
    state['next'] = 'Result_Validator'
    state['explanation'] = result['explanation']
    return state

def result_validator_node(state: LungCancerTStageState, agent):
    """
    결과 검증기: 최종 결정된 T 병기를 검증합니다.
    """
    result = agent.invoke(state)
    state['validation_result'] = result['output']
    state['next'] = 'END'
    state['explanation'] = result['explanation']
    return state

llm = ChatOpenAI(model="gpt-4o")

ct_agent = create_agent(llm, [], "You are a CT image analyzer specialized in lung cancer T staging.")
pet_agent = create_agent(llm, [], "You are a PET scan analyzer for lung cancer, focusing on metabolic activity and potential spread.")
size_classifier_agent = create_agent(llm, [], "You classify initial T stage based on tumor size.")
sync_multi_agent = create_agent(llm, [], "You specialize in identifying synchronous or multicentric cancer in lung cancer cases.")
main_bronchus_agent = create_agent(llm, [], "You specialize in identifying main bronchus involvement in lung cancer.")
invasion_agent = create_agent(llm, [], "You are specialized in identifying invasion of surrounding structures in lung cancer.")
t_classifier_agent = create_agent(llm, [], "You are an expert in classifying T stages of lung cancer based on all available information.")
year_checker_agent = create_agent(llm, [], "You determine whether to apply pre-2018 or post-2018 lung cancer staging criteria.")
final_t_determiner_agent = create_agent(llm, [], "You make the final determination of T stage in lung cancer, considering all factors and applicable criteria.")
result_validator_agent = create_agent(llm, [], "You validate the final T stage determination, ensuring all criteria have been correctly applied.")

supervisor_agent = create_agent(llm, [], "You are a supervisor reviewing the decisions and explanations of other agents in the lung cancer T staging process. Approve only if the explanation is logical and follows established guidelines.")

workflow = StateGraph(LungCancerTStageState)

workflow.add_node("CT_Analyzer", wrap_node_with_supervisor(functools.partial(ct_analyzer_node, agent=ct_agent), "CT_Analyzer"))
workflow.add_node("PET_Analyzer", wrap_node_with_supervisor(functools.partial(pet_analyzer_node, agent=pet_agent), "PET_Analyzer"))
workflow.add_node("Size_Based_Classifier", wrap_node_with_supervisor(functools.partial(size_based_classifier_node, agent=size_classifier_agent), "Size_Based_Classifier"))
workflow.add_node("Synchronous_Multicentric_Checker", wrap_node_with_supervisor(functools.partial(synchronous_multicentric_checker_node, agent=sync_multi_agent), "Synchronous_Multicentric_Checker"))
workflow.add_node("Main_Bronchus_Checker", wrap_node_with_supervisor(functools.partial(main_bronchus_checker_node, agent=main_bronchus_agent), "Main_Bronchus_Checker"))
workflow.add_node("Invasion_Checker", wrap_node_with_supervisor(functools.partial(invasion_checker_node, agent=invasion_agent), "Invasion_Checker"))
workflow.add_node("T_Stage_Classifier", wrap_node_with_supervisor(functools.partial(t_stage_classifier_node, agent=t_classifier_agent), "T_Stage_Classifier"))
workflow.add_node("Year_Checker", wrap_node_with_supervisor(functools.partial(year_checker_node, agent=year_checker_agent), "Year_Checker"))
workflow.add_node("Final_T_Stage_Determiner", wrap_node_with_supervisor(functools.partial(final_t_stage_determiner_node, agent=final_t_determiner_agent), "Final_T_Stage_Determiner"))
workflow.add_node("Result_Validator", wrap_node_with_supervisor(functools.partial(result_validator_node, agent=result_validator_agent), "Result_Validator"))

workflow.add_edge("CT_Analyzer", "PET_Analyzer")
workflow.add_edge("PET_Analyzer", "Size_Based_Classifier")
workflow.add_edge("Size_Based_Classifier", "Synchronous_Multicentric_Checker")
workflow.add_edge("Synchronous_Multicentric_Checker", "Main_Bronchus_Checker")
workflow.add_edge("Main_Bronchus_Checker", "Invasion_Checker")
workflow.add_edge("Invasion_Checker", "T_Stage_Classifier")
workflow.add_edge("T_Stage_Classifier", "Year_Checker")
workflow.add_edge("Year_Checker", "Final_T_Stage_Determiner")
workflow.add_edge("Final_T_Stage_Determiner", "Result_Validator")

workflow.add_conditional_edges(
    "Result_Validator",
    lambda x: x['next'],
    {'END': END}
)

workflow.set_entry_point("CT_Analyzer")

graph = workflow.compile()